In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [21]:
daily_act = pd.read_csv('./dailyActivity_merged.csv')
sleep_day = pd.read_csv('./sleepDay_merged.csv')
sleep_min = pd.read_csv('./minuteSleep_merged.csv')
step_min = pd.read_csv('./minuteStepsNarrow_merged.csv')
intens_min = pd.read_csv('./minuteIntensitiesNarrow_merged.csv')
heart_rate = pd.read_csv('./heartrate_seconds_merged.csv')
weight = pd.read_csv('./weightLogInfo_merged.csv')
calories_min = pd.read_csv('./minuteCaloriesNarrow_merged.csv')

In [22]:
daily_act = pd.read_csv('./dailyActivity_merged.csv')
sleep_day = pd.read_csv('./sleepDay_merged.csv')
sleep_min = pd.read_csv('./minuteSleep_merged.csv')
step_min = pd.read_csv('./minuteStepsNarrow_merged.csv')
intens_min = pd.read_csv('./minuteIntensitiesNarrow_merged.csv')
heart_rate = pd.read_csv('./heartrate_seconds_merged.csv')
weight = pd.read_csv('./weightLogInfo_merged.csv')
calories_min = pd.read_csv('./minuteCaloriesNarrow_merged.csv')

In [23]:
distinct_daily_act = daily_act['Id'].nunique()
distinct_sleep_day = sleep_day['Id'].nunique()
distinct_sleep_min = sleep_min['Id'].nunique()
distinct_step_min = step_min['Id'].nunique()
distinct_intens_min = intens_min['Id'].nunique()
distinct_heart_rate = heart_rate['Id'].nunique()
distinct_weight = weight['Id'].nunique()
distinct_calories_min = calories_min['Id'].nunique()


In [24]:
duplicates_daily_act = daily_act.duplicated(subset=['Id']).sum()
duplicates_sleep_day = sleep_day.duplicated(subset=['Id']).sum()
duplicates_sleep_min = sleep_min.duplicated(subset=['Id']).sum()
duplicates_step_min = step_min.duplicated(subset=['Id']).sum()
duplicates_intens_min = intens_min.duplicated(subset=['Id']).sum()
duplicates_heart_rate = heart_rate.duplicated(subset=['Id']).sum()
duplicates_weight = weight.duplicated(subset=['Id']).sum()
duplicates_calories_min = calories_min.duplicated(subset=['Id']).sum()

In [25]:
sleep_day = sleep_day.drop_duplicates(subset=['Id'])
sleep_min = sleep_min.drop_duplicates(subset=['Id'])

In [26]:
Id_days = daily_act.groupby('Id').size().reset_index(name='days')

Id_days['category'] = pd.cut(Id_days['days'],
                             bins=[0, 10, 20, 31],
                             labels=["low use", "moderate use", "high use"])

Id_days_new = Id_days.groupby('category').size().reset_index(name='user_number')

print(Id_days_new.head())

       category  user_number
0       low use            1
1  moderate use            3
2      high use           29


In [28]:
import pandas as pd


daily_act['ActivityDate'] = pd.to_datetime(daily_act['ActivityDate'], format='%m/%d/%Y')

print("Column names of sleep_day:", sleep_day.columns)

sleep_day1 = sleep_day[['SleepDay']].copy()
sleep_day1[['ActivityDate', 'time', 'am_pm']] = sleep_day1['SleepDay'].str.split(' ', expand=True)
sleep_day1['ActivityDate'] = pd.to_datetime(sleep_day1['ActivityDate'], format='%m/%d/%Y')

sleep_day1 = pd.merge(sleep_day1, daily_act[['Id', 'ActivityDate']], on='ActivityDate', how='left')

full_day = pd.merge(daily_act, sleep_day1, on=['Id', 'ActivityDate'], how='outer')

full_day = full_day.fillna(0)

print(full_day.head())

Column names of sleep_day: Index(['Id', 'SleepDay', 'TotalSleepRecords', 'TotalMinutesAsleep',
       'TotalTimeInBed'],
      dtype='object')
           Id ActivityDate  TotalSteps  TotalDistance  TrackerDistance  \
0  1503960366   2016-04-12       13162            8.5              8.5   
1  1503960366   2016-04-12       13162            8.5              8.5   
2  1503960366   2016-04-12       13162            8.5              8.5   
3  1503960366   2016-04-12       13162            8.5              8.5   
4  1503960366   2016-04-12       13162            8.5              8.5   

   LoggedActivitiesDistance  VeryActiveDistance  ModeratelyActiveDistance  \
0                       0.0                1.88                      0.55   
1                       0.0                1.88                      0.55   
2                       0.0                1.88                      0.55   
3                       0.0                1.88                      0.55   
4                       0.0

In [27]:
sleep_min = sleep_min.copy()

sleep_min['minutes'] = sleep_min['value']

sleep_min.loc[sleep_min['minutes'] != 0, 'minutes'] = 1

print(sleep_min.describe())

                 Id      value         logId  minutes
count  2.400000e+01  24.000000  2.400000e+01     24.0
mean   4.763820e+09   1.875000  1.140184e+10      1.0
std    2.288233e+09   0.797414  3.987637e+07      0.0
min    1.503960e+09   1.000000  1.137223e+10      1.0
25%    2.340408e+09   1.000000  1.137484e+10      1.0
50%    4.501862e+09   2.000000  1.138018e+10      1.0
75%    6.822462e+09   2.250000  1.141987e+10      1.0
max    8.792010e+09   3.000000  1.152362e+10      1.0


In [29]:

sleep_min[['SleepDate', 'time', 'am_pm']] = sleep_min['date'].str.split(' ', expand=True)
sleep_min['SleepDate'] = pd.to_datetime(sleep_min['SleepDate'], format='%m/%d/%Y')

sleep_min1 = sleep_min.groupby(['Id', 'SleepDate']).agg(total_sleep=('minutes', 'sum')).reset_index()

full_day_new = pd.merge(full_day, sleep_min1, left_on=['Id', 'ActivityDate'], right_on=['Id', 'SleepDate'], how='outer')

full_day_new = full_day_new.fillna(0)

print(full_day_new.head())

           Id         ActivityDate  TotalSteps  TotalDistance  \
0  1503960366  2016-04-12 00:00:00     13162.0            8.5   
1  1503960366  2016-04-12 00:00:00     13162.0            8.5   
2  1503960366  2016-04-12 00:00:00     13162.0            8.5   
3  1503960366  2016-04-12 00:00:00     13162.0            8.5   
4  1503960366  2016-04-12 00:00:00     13162.0            8.5   

   TrackerDistance  LoggedActivitiesDistance  VeryActiveDistance  \
0              8.5                       0.0                1.88   
1              8.5                       0.0                1.88   
2              8.5                       0.0                1.88   
3              8.5                       0.0                1.88   
4              8.5                       0.0                1.88   

   ModeratelyActiveDistance  LightActiveDistance  SedentaryActiveDistance  \
0                      0.55                 6.06                      0.0   
1                      0.55                 6.

In [30]:
full_day_new['total_time_check'] = (full_day_new['VeryActiveMinutes'] +
                                    full_day_new['FairlyActiveMinutes'] +
                                    full_day_new['LightlyActiveMinutes'] +
                                    full_day_new['SedentaryMinutes'] +
                                    full_day_new['total_sleep']) / 60

# Filter on days that are more than 24 hours
full_day_check3 = full_day_new[full_day_new['total_time_check'] > 24]

# Display information for full_day_check3
print("Full Day Check3:")
print(full_day_check3.head())

# Filter on days that are less than 24 hours
full_day_check4 = full_day_new[full_day_new['total_time_check'] < 24]

# Display information for full_day_check4
print("\nFull Day Check4:")
print(full_day_check4.head())

Full Day Check3:
Empty DataFrame
Columns: [Id, ActivityDate, TotalSteps, TotalDistance, TrackerDistance, LoggedActivitiesDistance, VeryActiveDistance, ModeratelyActiveDistance, LightActiveDistance, SedentaryActiveDistance, VeryActiveMinutes, FairlyActiveMinutes, LightlyActiveMinutes, SedentaryMinutes, Calories, SleepDay, time, am_pm, SleepDate, total_sleep, total_time_check]
Index: []

[0 rows x 21 columns]

Full Day Check4:
           Id         ActivityDate  TotalSteps  TotalDistance  \
0  1503960366  2016-04-12 00:00:00     13162.0            8.5   
1  1503960366  2016-04-12 00:00:00     13162.0            8.5   
2  1503960366  2016-04-12 00:00:00     13162.0            8.5   
3  1503960366  2016-04-12 00:00:00     13162.0            8.5   
4  1503960366  2016-04-12 00:00:00     13162.0            8.5   

   TrackerDistance  LoggedActivitiesDistance  VeryActiveDistance  \
0              8.5                       0.0                1.88   
1              8.5                       0.0

In [32]:
average_sleep = full_day_new.groupby('Id').agg(
    mean_sleep=('total_sleep', 'mean'),
    mean_sedentary=('SedentaryMinutes', 'mean')
).reset_index()

average_sleep['mean_sleep'] /= 60
average_sleep['mean_sedentary'] /= 60

print(average_sleep.head())

           Id  mean_sleep  mean_sedentary
0  1503960366    0.004710       13.515942
1  1624580081    0.000000       21.165942
2  1644430081    0.000370       19.290741
3  1844505072    0.000362       19.430797
4  1927972279    0.000355       18.799645


In [33]:
# checking download 
print(step_min.head())
print(intens_min.head())
print(calories_min.head())
print(heart_rate.head())

           Id         ActivityMinute  Steps
0  1503960366  4/12/2016 12:00:00 AM      0
1  1503960366  4/12/2016 12:01:00 AM      0
2  1503960366  4/12/2016 12:02:00 AM      0
3  1503960366  4/12/2016 12:03:00 AM      0
4  1503960366  4/12/2016 12:04:00 AM      0
           Id         ActivityMinute  Intensity
0  1503960366  4/12/2016 12:00:00 AM          0
1  1503960366  4/12/2016 12:01:00 AM          0
2  1503960366  4/12/2016 12:02:00 AM          0
3  1503960366  4/12/2016 12:03:00 AM          0
4  1503960366  4/12/2016 12:04:00 AM          0
           Id         ActivityMinute  Calories
0  1503960366  4/12/2016 12:00:00 AM    0.7865
1  1503960366  4/12/2016 12:01:00 AM    0.7865
2  1503960366  4/12/2016 12:02:00 AM    0.7865
3  1503960366  4/12/2016 12:03:00 AM    0.7865
4  1503960366  4/12/2016 12:04:00 AM    0.7865
           Id                  Time  Value
0  2022484408  4/12/2016 7:21:00 AM     97
1  2022484408  4/12/2016 7:21:05 AM    102
2  2022484408  4/12/2016 7:21:10 AM  

In [34]:
weight[['WeightDate', 'time', 'am_pm']] = weight['Date'].str.split(' ', expand=True)
weight['WeightDate'] = pd.to_datetime(weight['WeightDate'], format='%m/%d/%Y')
weight['time'] = pd.to_datetime(weight['time'], format='%H:%M:%S').dt.time





In [35]:
calories_min[['ActivityDate', 'time', 'am_pm']] = calories_min['ActivityMinute'].str.split(' ', expand=True)
calories_min['ActivityDate'] = pd.to_datetime(calories_min['ActivityDate'], format='%m/%d/%Y')
calories_min['time'] = pd.to_datetime(calories_min['time'], format='%H:%M:%S').dt.time

In [46]:
split_time = heart_rate['Time'].str.split(' ', expand=True)

# Assign separate columns to heart_rate
heart_rate['ActivityDate'] = pd.to_datetime(split_time[0], format='%m/%d/%Y', errors='coerce')
heart_rate['time'] = pd.to_datetime(split_time[1], errors='coerce').dt.time

/tmp/ipykernel_63295/514664688.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  heart_rate['time'] = pd.to_datetime(split_time[1], errors='coerce').dt.time


In [40]:
# Separate 'Time' column in heart_rate
split_time = heart_rate['Time'].str.split(' ', expand=True)

# Assign separate columns to heart_rate
heart_rate['ActivityDate'] = pd.to_datetime(split_time[0], format='%m/%d/%Y', errors='coerce')
heart_rate['time'] = pd.to_datetime(split_time[1], format='%H:%M:%S', errors='coerce').dt.time


In [56]:
intensity_steps['ActivityDate'] = pd.to_datetime(intensity_steps['ActivityDate'])

# Now you can proceed with the merging
full_data = pd.merge(heart_rate, calories_min, on=['Id', 'ActivityDate', 'time'], how='outer')
full_data = pd.merge(full_data, intensity_steps, on=['Id', 'ActivityDate'], how='outer')

# Continue with the merging of other DataFrames (e.g., weight) if needed
# full_data = pd.merge(full_data, weight, on=['Id', 'ActivityDate', 'time'], how='outer')

# Print or inspect the resulting DataFrame
print(full_data.head())

           Id                  Time  Value ActivityDate      time  \
0  2022484408  4/12/2016 7:21:00 AM   97.0   2016-04-12  07:21:00   
1  2022484408  4/12/2016 7:21:00 AM   97.0   2016-04-12  07:21:00   
2  2022484408  4/12/2016 7:21:05 AM  102.0   2016-04-12  07:21:05   
3  2022484408  4/12/2016 7:21:10 AM  105.0   2016-04-12  07:21:10   
4  2022484408  4/12/2016 7:21:10 PM   74.0   2016-04-12  07:21:10   

         ActivityMinute  Calories am_pm  TotalSteps  
0  4/12/2016 7:21:00 AM   3.32064    AM         NaN  
1  4/12/2016 7:21:00 PM   1.14147    PM         NaN  
2                   NaN       NaN   NaN         NaN  
3                   NaN       NaN   NaN         NaN  
4                   NaN       NaN   NaN         NaN  


In [57]:
print("Column names of heart_rate:", heart_rate.columns)
print("Column names of calories_min:", calories_min.columns)
print("Column names of intensity_steps:", intensity_steps.columns)
print("Column names of weight:", weight.columns)

Column names of heart_rate: Index(['Id', 'Time', 'Value', 'ActivityDate', 'time'], dtype='object')
Column names of calories_min: Index(['Id', 'ActivityMinute', 'Calories', 'ActivityDate', 'time', 'am_pm'], dtype='object')
Column names of intensity_steps: Index(['Id', 'ActivityDate', 'TotalSteps'], dtype='object')
Column names of weight: Index(['Id', 'Date', 'WeightKg', 'WeightPounds', 'Fat', 'BMI',
       'IsManualReport', 'LogId', 'WeightDate', 'time', 'am_pm'],
      dtype='object')


In [61]:
intensity_steps.loc[(intensity_steps['TotalSteps'] == 0) & 'TotalSteps'] = 1


KeyError: 'Intensity'

In [62]:
intensity_steps

,Id,ActivityDate,TotalSteps
0,1,2023-01-01,5000
1,2,2023-01-02,7000
2,3,2023-01-03,6000
